In [1]:
import os
os.getcwd()

'/Users/yuechenjiang/Desktop/CatMemo/Final_result'

In [2]:
cd /Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/

/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

# Get daily stock price
def get_price(Start:str, End:str, Ticker):
    """
    Fetch daily adjusted closing prices of a stock from Yahoo Finance.

    Parameters:
    Start (str): Start date for the price data.
    End (str): End date for the price data.
    Ticker (str): Ticker symbol of the stock.

    Returns:
    list: List of daily adjusted closing prices.
    """
    df = yf.download(ticker, start=Start, end=End)
    return df['Adj Close'].tolist()

# Get actions for different models
def get_action(start, end, ticker, file_path, col):
    """
    Fetch action data from a specified CSV file.

    Parameters:
    start (str): Start date for the action data.
    end (str): End date for the action data.
    ticker (str): Ticker symbol of the stock (not used in the function).
    file_path (str): Path to the CSV file containing the action data.
    col (list): Column names to be used for date and action.

    Returns:
    list: List of actions corresponding to the specified date range.
    """
    df = pd.read_csv(file_path)
    df[col[0]] = pd.to_datetime(df[col[0]])
    action = df[(df[col[0]] >= pd.to_datetime(start)) & (df[col[0]] < pd.to_datetime(end))][col[1]].tolist()
    return action

def daily_reward(price, actions):
    """
    Calculate daily rewards based on price changes and actions.

    Parameters:
    price (list): List of prices.
    actions (list): List of actions taken.

    Returns:
    list: Daily rewards.
    """
    reward = []
    for i in range(len(price) - 1):
        r = actions[i] * np.log(price[i+1] / price[i])
        reward.append(r)
    return reward

def standard_deviation(reward):
    """
    Calculate the daily standard deviation of rewards.

    Parameters:
    reward (list): List of rewards.

    Returns:
    float: Standard deviation of the rewards.
    """
    mean = sum(reward) / len(reward)
    variance = sum((r - mean) ** 2 for r in reward) / (len(reward) - 1)
    return variance ** 0.5

def total_reward(price, actions):
    """
    Calculate the cumulative return over the trading period.

    Parameters:
    price (list): List of prices.
    actions (list): List of actions taken.

    Returns:
    float: Total cumulative reward.
    """
    reward = 0
    for i in range(len(price) - 1):
        reward += actions[i] * np.log(price[i+1] / price[i])
    return reward

def annualized_volatility(daily_std_dev, trading_days=252):
    """
    Calculate the annualized volatility from daily standard deviation.

    Parameters:
    daily_std_dev (float): Standard deviation of daily returns.
    trading_days (int): Number of trading days in a year, typically 252.

    Returns:
    float: Annualized volatility.
    """
    return daily_std_dev * (trading_days ** 0.5)

def calculate_sharpe_ratio(Rp, Rf, sigma_p, price):
    """
    Calculate the Sharpe ratio of an investment.

    Parameters:
    Rp (float): Return of the portfolio.
    Rf (float): Risk-free rate (annualized).
    sigma_p (float): Standard deviation of the portfolio's excess return.

    Returns:
    float: Sharpe ratio.
    """
    if sigma_p == 0:
        raise ValueError("Standard deviation cannot be zero.")
    Rp = Rp / (len(price)/252)
    
    return (Rp - Rf) / sigma_p

def calculate_max_drawdown(daily_returns):
    """
    Calculate the maximum drawdown of a portfolio.

    Parameters:
    daily_returns (list): List of daily returns.

    Returns:
    float: Maximum drawdown.
    """
    cumulative_returns = [1]
    for r in daily_returns:
        cumulative_returns.append(cumulative_returns[-1] * (1 + r))

    peak = cumulative_returns[0]
    max_drawdown = 0

    for r in cumulative_returns:
        if r > peak:
            peak = r
        drawdown = (peak - r) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    return max_drawdown
    
def calculate_metrics(price, actions):
    """
    Calculate various financial metrics based on price and actions.

    Parameters:
    price (list): List of daily prices.
    actions (list): List of actions taken.

    Returns:
    tuple: A tuple containing calculated metrics (standard deviation, annualized volatility, cumulative return, Sharpe ratio, max drawdown).
    """
    daily_rw = daily_reward(price, actions)
    std_dev_r = standard_deviation(daily_rw)
    
    
    ann_vol = annualized_volatility(std_dev_r)
    cum_return = total_reward(price, actions)
    sharpe_ratio = calculate_sharpe_ratio(cum_return, 0, ann_vol, price)
    max_dd = calculate_max_drawdown(daily_rw)
    return cum_return, sharpe_ratio, std_dev_r, ann_vol, max_dd

def main(ticker, start, end, df_paths, col_names, save_path):
    """
    Main function to calculate metrics and save results to a CSV file.

    Parameters:
    ticker (str): Ticker symbol of the stock.
    start (str): Start date for analysis.
    end (str): End date for analysis.
    df_paths (dict): Dictionary of file paths for different models.
    col_names (dict): Dictionary of column names for different models.
    save_path (str): Path to save the results CSV file.
    """
    price = get_price(start, end, ticker)
    delta = max(price)-min(price)
    print('The price range of {0} in test period is {1}'.format(ticker, round(delta, 2)))
    metrics = ['Cumulative Return', 'Sharpe Ratio', 'Standard Deviation', 'Annualized Volatility', 'Max Drawdown']
    results = {'Buy & Hold': calculate_metrics(price, [1] * len(price))}
    daily_price_change_percentage = list(map(lambda p: (p[1] - p[0]) / p[0], zip(price[:-1], price[1:])))
    std_dev_p = standard_deviation(daily_price_change_percentage)
    for model, file_path in df_paths.items():
        if model == 'FinMe':
            print('Standard Deviation of Daily Price Change Percentage of {0}: {1:.2%}'.format(ticker, std_dev_p))
        col = col_names[model]
        actions = get_action(start, end, ticker, file_path, col)
        results[model] = calculate_metrics(price, actions)

    df_results = pd.DataFrame(results, index=metrics)
    df_results.to_csv(save_path)
    print(df_results)

In [4]:
if __name__ == '__main__':
    ticker = 'TSLA'
    start_time = '2022-06-25'
    end_time = '2023-04-25'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/Tsla-new-full.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_tsla_park_v2.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/tsla_curie.csv',
        'A2C': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_A2C_summary_data_seed2_full.csv',
        'PPO': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_PPO_summary_data_seed1_full.csv',
        'DQN': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_DQN_summary_data_seed1_full.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions'],
        'A2C': ['date', 'action'],
        'PPO': ['date', 'action'],
        'DQN': ['date', 'action']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/TSLA2.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed
The price range of TSLA in test period is 201.22
Standard Deviation of Daily Price Change Percentage of TSLA: 3.97%
                       Buy & Hold     FinMe      Park    FinGPT       A2C  \
Cumulative Return       -0.409946  0.779307  0.135213 -0.015201 -0.057557   
Sharpe Ratio            -0.784341  2.149513  0.388944 -0.035811 -0.109988   
Standard Deviation       0.039890  0.027670  0.026532  0.032395  0.039938   
Annualized Volatility    0.633226  0.439244  0.421181  0.514251  0.633995   
Max Drawdown             0.680383  0.125734  0.347150  0.423993  0.677667   

                            PPO       DQN  
Cumulative Return     -0.210757  0.025470  
Sharpe Ratio          -0.402870  0.048672  
Standard Deviation     0.039926  0.039939  
Annualized Volatility  0.633803  0.634007  
Max Drawdown           0.673269  0.663544  


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [5]:
if __name__ == '__main__':
    ticker = 'TSLA'
    start_time = '2022-10-06'
    end_time = '2023-04-10'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/Tsla-new-full.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_tsla_park_v2.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/tsla_curie.csv',
        'A2C': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_A2C_summary_data_seed2_full.csv',
        'PPO': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_PPO_summary_data_seed1_full.csv',
        'DQN': '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_DQN_summary_data_seed1_full.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions'],
        'A2C': ['date', 'action'],
        'PPO': ['date', 'action'],
        'DQN': ['date', 'action']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/TSLA.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed
The price range of TSLA in test period is 120.42
Standard Deviation of Daily Price Change Percentage of TSLA: 4.40%


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


                       Buy & Hold     FinMe      Park    FinGPT       A2C  \
Cumulative Return       -0.186312  0.617758  0.134636 -0.074554  0.137067   
Sharpe Ratio            -0.541048  2.678859  0.599015 -0.279530  0.397932   
Standard Deviation       0.044084  0.029522  0.028774  0.034145  0.044096   
Annualized Volatility    0.699818  0.468649  0.456774  0.542027  0.700009   
Max Drawdown             0.553208  0.107996  0.243177  0.423993  0.523308   

                            PPO       DQN  
Cumulative Return      0.012877  0.333393  
Sharpe Ratio           0.037372  0.969439  
Standard Deviation     0.044110  0.044027  
Annualized Volatility  0.700232  0.698900  
Max Drawdown           0.543264  0.520033  


In [6]:
if __name__ == '__main__':
    ticker = 'NFLX'
    start_time = '2022-10-06'
    end_time = '2023-04-10'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_nflx_new_v3.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX111.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/nflx_curie.csv',
        'A2C': '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_A2C_summary_data_seed0.csv',
        'PPO': '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_PPO_summary_data_seed2.csv',
        'DQN': '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_DQN_summary_data_seed1.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions'],
        'A2C': ['date', 'action'],
        'PPO': ['date', 'action'],
        'DQN': ['date', 'action']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/NFLX.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed
The price range of NFLX in test period is 153.67
Standard Deviation of Daily Price Change Percentage of NFLX: 3.24%


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


                       Buy & Hold     FinMe      Park    FinGPT       A2C  \
Cumulative Return        0.355111  0.364485  0.320058  0.090090  0.146155   
Sharpe Ratio             1.410901  2.016790  1.596455  0.426610  0.578750   
Standard Deviation       0.031964  0.022951  0.025460  0.026819  0.032071   
Annualized Volatility    0.507410  0.364342  0.404168  0.425732  0.509112   
Max Drawdown             0.209263  0.158495  0.169893  0.282705  0.250184   

                            PPO       DQN  
Cumulative Return      0.084121 -0.122067  
Sharpe Ratio           0.332954 -0.483267  
Standard Deviation     0.032086  0.032078  
Annualized Volatility  0.509344  0.509217  
Max Drawdown           0.250184  0.287017  


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [7]:
if __name__ == '__main__':
    ticker = 'AMZN'
    start_time = '2022-10-06'
    end_time = '2023-04-10'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_amzn_new_v5.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN111.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/amzn_curie-001.csv',
        'A2C': '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_A2C_summary_data_seed0.csv',
        'PPO': '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_PPO_summary_data_seed2_new.csv',
        'DQN': '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_DQN_summary_data_seed2.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions'],
        'A2C': ['date', 'action'],
        'PPO': ['date', 'action'],
        'DQN': ['date', 'action']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/AMZN.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed

/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])



The price range of AMZN in test period is 38.78
Standard Deviation of Daily Price Change Percentage of AMZN: 2.79%
                       Buy & Hold     FinMe      Park    FinGPT       A2C  \
Cumulative Return       -0.107739  0.048850 -0.139271 -0.296781 -0.063591   
Sharpe Ratio            -0.497988  0.232728 -0.998092 -2.175617 -0.293831   
Standard Deviation       0.027697  0.026872  0.017864  0.017464  0.027706   
Annualized Volatility    0.439674  0.426576  0.283576  0.277225  0.439819   
Max Drawdown             0.336828  0.229294  0.277334  0.284838  0.261275   

                            PPO       DQN  
Cumulative Return     -0.084194 -0.299820  
Sharpe Ratio          -0.389085 -1.390561  
Standard Deviation     0.027702  0.027603  
Annualized Volatility  0.439761  0.438177  
Max Drawdown           0.336828  0.383740  


In [8]:
if __name__ == '__main__':
    ticker = 'MSFT'
    start_time = '2022-10-06'
    end_time = '2023-04-10'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_msft_renewed_seeking_v8.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_msft_v2_park.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/msft_curie_001.csv',
        'A2C': '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_A2C_summary_data_full.csv',
        'PPO': '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_PPO_summary_data_2.csv',
        'DQN': '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_DQN_summary_data.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions'],
        'A2C': ['date', 'action'],
        'PPO': ['date', 'action'],
        'DQN': ['date', 'action']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/MSFT.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed
The price range of MSFT in test period is 74.71
Standard Deviation of Daily Price Change Percentage of MSFT: 2.24%


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


                       Buy & Hold     FinMe      Park    FinGPT       A2C  \
Cumulative Return        0.146949  0.232613 -0.181031  0.057356  0.004599   
Sharpe Ratio             0.835890  1.440183 -1.605733  0.443015  0.026121   
Standard Deviation       0.022326  0.020512  0.014318  0.016442  0.022357   
Annualized Volatility    0.354411  0.325617  0.227285  0.261008  0.354913   
Max Drawdown             0.150097  0.149889  0.242074  0.128459  0.236781   

                            PPO       DQN  
Cumulative Return      0.128066  0.147397  
Sharpe Ratio           0.728233  0.838449  
Standard Deviation     0.022333  0.022326  
Annualized Volatility  0.354532  0.354408  
Max Drawdown           0.195355  0.251844  


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [9]:
if __name__ == '__main__':
    ticker = 'COIN'
    start_time = '2022-10-06'
    end_time = '2023-04-10'
    
    df_paths = {
        'FinMe': '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_coin_v2.csv',
        'Park': '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_coin_park_v2.csv',
        'FinGPT': '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/coin_curie.csv'
    }

    col_names = {
        'FinMe': ['date', 'direction'],
        'Park': ['date', 'direction'],
        'FinGPT': ['dates', 'actions']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/COIN.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed

/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])



The price range of COIN in test period is 51.46
Standard Deviation of Daily Price Change Percentage of COIN: 6.85%
                       Buy & Hold     FinMe      Park    FinGPT
Cumulative Return       -0.300071  0.349832  0.034627 -0.887805
Sharpe Ratio            -0.514982  0.716965  0.089594 -1.950709
Standard Deviation       0.067517  0.056538  0.044783  0.052736
Annualized Volatility    1.071795  0.897515  0.710908  0.837153
Max Drawdown             0.605084  0.357526  0.320957  0.735774


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [10]:
if __name__ == '__main__':
    ticker = 'TSLA'
    start_time = '2022-06-16'
    end_time = '2022-12-28'
    
    df_paths = {
        'Self Adaptive': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/character/Tsla-new-full.csv',
        'Risk Seeking': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/character/TSLA_seeking_V8.csv',
        'Risk Averse': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/character/TSLA_averse_V6.csv'
    }

    col_names = {
        'Self Adaptive': ['date', 'direction'],
        'Risk Seeking': ['date', 'direction'],
        'Risk Averse': ['date', 'direction']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/character.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed
The price range of TSLA in test period is 200.22


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


                       Buy & Hold  Self Adaptive  Risk Seeking  Risk Averse
Cumulative Return       -0.669497       0.546958     -0.194132    -0.124679
Sharpe Ratio            -2.084456       2.496017     -0.786630    -1.578318
Standard Deviation       0.038050       0.025960      0.029236     0.009358
Annualized Volatility    0.604020       0.412100      0.464112     0.148557
Max Drawdown             0.673269       0.125734      0.450001     0.159882


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [11]:
if __name__ == '__main__':
    ticker = 'TSLA'
    start_time = '2022-06-16'
    end_time = '2022-12-28'
    
    df_paths = {
        'Top 1': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/TopK/TSLA_top1.csv',
        'Top 3': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/TopK/TSLA_top3.csv',
        'Top 5': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/character/Tsla-new-full.csv',
        'Top 10': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/TopK/TSLA_top10.csv',
    }

    col_names = {
        'Top 1': ['date', 'direction'],
        'Top 3': ['date', 'direction'],
        'Top 5': ['date', 'direction'],
        'Top 10': ['date', 'direction']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/TopK.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed

/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])



The price range of TSLA in test period is 200.22
                       Buy & Hold     Top 1     Top 3     Top 5    Top 10
Cumulative Return       -0.669497  0.520936  0.294430  0.546958  0.794448
Sharpe Ratio            -2.084456  1.864161  1.121361  2.496017  2.746943
Standard Deviation       0.038050  0.033105  0.031105  0.025960  0.034262
Annualized Volatility    0.604020  0.525529  0.493779  0.412100  0.543891
Max Drawdown             0.673269  0.252355  0.270972  0.125734  0.171360


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [12]:
if __name__ == '__main__':
    ticker = 'TSLA'
    start_time = '2022-06-16'
    end_time = '2022-12-28'
    
    df_paths = {
        'ChatGPT3.5-Turbo': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/change_LLM/dat_df_GPT3.5_turbo.csv',
        'ChatGPT4': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/change_LLM/TSLA_GPT4.csv',
        'ChatGPT4-Turbo': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/character/Tsla-new-full.csv',
        'davinci-003': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/change_LLM/dat_df_davicin003.csv',
        'Llama-70b-chat': '/Users/yuechenjiang/Desktop/CatMemo/Final_result/change_LLM/dat_df_llama.csv'
    }

    col_names = {
        'ChatGPT3.5-Turbo': ['date', 'direction'],
        'ChatGPT4': ['date', 'direction'],
        'ChatGPT4-Turbo': ['date', 'direction'],
        'davinci-003': ['date', 'direction'],
        'Llama-70b-chat': ['date', 'direction']
    }

    save_path = '/Users/yuechenjiang/Desktop/CatMemo/Final_result/metrics/LLM.csv'
    
    main(ticker, start_time, end_time, df_paths, col_names, save_path)

[*********************100%%**********************]  1 of 1 completed

/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])



The price range of TSLA in test period is 200.22
                       Buy & Hold  ChatGPT3.5-Turbo  ChatGPT4  ChatGPT4-Turbo  \
Cumulative Return       -0.669497          0.161501  0.626180        0.546958   
Sharpe Ratio            -2.084456          2.158881  2.225073        2.496017   
Standard Deviation       0.038050          0.008862  0.033339        0.025960   
Annualized Volatility    0.604020          0.140683  0.529237        0.412100   
Max Drawdown             0.673269          0.011073  0.174012        0.125734   

                       davinci-003  Llama-70b-chat  
Cumulative Return         0.016308       -0.527233  
Sharpe Ratio              0.851477       -2.853249  
Standard Deviation        0.002269        0.021891  
Annualized Volatility     0.036018        0.347503  
Max Drawdown              0.008408        0.447168  


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/1493465817.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_92937/14934658

In [13]:
import pandas as pd

# Load the CSV files
tsla_df = pd.read_csv('TSLA.csv', index_col=0)
nflx_df = pd.read_csv('NFLX.csv', index_col=0)
amzn_df = pd.read_csv('AMZN.csv', index_col=0)
msft_df = pd.read_csv('MSFT.csv', index_col=0)
coin_df = pd.read_csv('COIN.csv', index_col=0)

# Rename columns to include the ticker name
tsla_df.columns = ['TSLA,' + col for col in tsla_df.columns]
nflx_df.columns = ['NFLX,' + col for col in nflx_df.columns]
amzn_df.columns = ['AMZN,' + col for col in amzn_df.columns]
msft_df.columns = ['MSFT,' + col for col in msft_df.columns]
coin_df.columns = ['COIN,' + col for col in coin_df.columns]

# Combine the dataframes
combined_df = pd.concat([tsla_df, nflx_df, amzn_df, msft_df, coin_df], axis=1)
transposed_df = combined_df.T
# rounded_df = transposed_df.round(3)
transposed_df.to_csv('combined_table.csv')

In [14]:
combined_table_df = pd.read_csv('combined_table.csv')

In [15]:
# Let's read the CSV file again and split the 'Unnamed: 0' column into 'ticker' and 'model' columns

combined_table_path = 'combined_table.csv'
combined_table_df = pd.read_csv(combined_table_path)

# Splitting the 'Unnamed: 0' column into 'ticker' and 'model'
combined_table_df[['ticker', 'model']] = combined_table_df['Unnamed: 0'].str.split(',', expand=True)

# Dropping the original 'Unnamed: 0' column
combined_table_df.drop('Unnamed: 0', axis=1, inplace=True)

# Rearranging columns so that 'ticker' and 'model' are the first two columns
column_order = ['ticker', 'model'] + [col for col in combined_table_df if col not in ['ticker', 'model']]
combined_table_df = combined_table_df[column_order]

combined_table_df  # Display the first few rows for verification

,ticker,model,Cumulative Return,Sharpe Ratio,Standard Deviation,Annualized Volatility,Max Drawdown
0,TSLA,Buy & Hold,-0.186312,-0.541048,0.044084,0.699818,0.553208
1,TSLA,FinMe,0.617758,2.678859,0.029522,0.468649,0.107996
2,TSLA,Park,0.134636,0.599015,0.028774,0.456774,0.243177
3,TSLA,FinGPT,-0.074554,-0.279530,0.034145,0.542027,0.423993
4,TSLA,A2C,0.137067,0.397932,0.044096,0.700009,0.523308
5,TSLA,PPO,0.012877,0.037372,0.044110,0.700232,0.543264
6,TSLA,DQN,0.333393,0.969439,0.044027,0.698900,0.520033
7,NFLX,Buy & Hold,0.355111,1.410901,0.031964,0.507410,0.209263
8,NFLX,FinMe,0.364485,2.016790,0.022951,0.364342,0.158495
9,NFLX,Park,0.320058,1.596455,0.025460,0.404168,0.169893


In [16]:
# Reset the index of the DataFrame
reset_df = combined_table_df.reset_index(drop=True)
# reset_df = reset_df.round(3)
# Generating LaTeX code for the adjusted table
latex_table_no_index = reset_df.to_latex(index=False)
latex_table_no_index

'\\begin{tabular}{llrrrrr}\n\\toprule\nticker & model & Cumulative Return & Sharpe Ratio & Standard Deviation & Annualized Volatility & Max Drawdown \\\\\n\\midrule\nTSLA & Buy & Hold & -0.186312 & -0.541048 & 0.044084 & 0.699818 & 0.553208 \\\\\nTSLA & FinMe & 0.617758 & 2.678859 & 0.029522 & 0.468649 & 0.107996 \\\\\nTSLA & Park & 0.134636 & 0.599015 & 0.028774 & 0.456774 & 0.243177 \\\\\nTSLA & FinGPT & -0.074554 & -0.279530 & 0.034145 & 0.542027 & 0.423993 \\\\\nTSLA & A2C & 0.137067 & 0.397932 & 0.044096 & 0.700009 & 0.523308 \\\\\nTSLA & PPO & 0.012877 & 0.037372 & 0.044110 & 0.700232 & 0.543264 \\\\\nTSLA & DQN & 0.333393 & 0.969439 & 0.044027 & 0.698900 & 0.520033 \\\\\nNFLX & Buy & Hold & 0.355111 & 1.410901 & 0.031964 & 0.507410 & 0.209263 \\\\\nNFLX & FinMe & 0.364485 & 2.016790 & 0.022951 & 0.364342 & 0.158495 \\\\\nNFLX & Park & 0.320058 & 1.596455 & 0.025460 & 0.404168 & 0.169893 \\\\\nNFLX & FinGPT & 0.090090 & 0.426610 & 0.026819 & 0.425732 & 0.282705 \\\\\nNFLX & A2C &

In [17]:
TopK = pd.read_csv('TopK.csv')
TopK.to_latex(index=False)

'\\begin{tabular}{lrrrrr}\n\\toprule\nUnnamed: 0 & Buy & Hold & Top 1 & Top 3 & Top 5 & Top 10 \\\\\n\\midrule\nCumulative Return & -0.669497 & 0.520936 & 0.294430 & 0.546958 & 0.794448 \\\\\nSharpe Ratio & -2.084456 & 1.864161 & 1.121361 & 2.496017 & 2.746943 \\\\\nStandard Deviation & 0.038050 & 0.033105 & 0.031105 & 0.025960 & 0.034262 \\\\\nAnnualized Volatility & 0.604020 & 0.525529 & 0.493779 & 0.412100 & 0.543891 \\\\\nMax Drawdown & 0.673269 & 0.252355 & 0.270972 & 0.125734 & 0.171360 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [18]:
LLM = pd.read_csv('LLM.csv')
LLM.to_latex(index=False)

'\\begin{tabular}{lrrrrrr}\n\\toprule\nUnnamed: 0 & Buy & Hold & ChatGPT3.5-Turbo & ChatGPT4 & ChatGPT4-Turbo & davinci-003 & Llama-70b-chat \\\\\n\\midrule\nCumulative Return & -0.669497 & 0.161501 & 0.626180 & 0.546958 & 0.016308 & -0.527233 \\\\\nSharpe Ratio & -2.084456 & 2.158881 & 2.225073 & 2.496017 & 0.851477 & -2.853249 \\\\\nStandard Deviation & 0.038050 & 0.008862 & 0.033339 & 0.025960 & 0.002269 & 0.021891 \\\\\nAnnualized Volatility & 0.604020 & 0.140683 & 0.529237 & 0.412100 & 0.036018 & 0.347503 \\\\\nMax Drawdown & 0.673269 & 0.011073 & 0.174012 & 0.125734 & 0.008408 & 0.447168 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [19]:
character = pd.read_csv('character.csv')
character.to_latex(index=False)

'\\begin{tabular}{lrrrr}\n\\toprule\nUnnamed: 0 & Buy & Hold & Self Adaptive & Risk Seeking & Risk Averse \\\\\n\\midrule\nCumulative Return & -0.669497 & 0.546958 & -0.194132 & -0.124679 \\\\\nSharpe Ratio & -2.084456 & 2.496017 & -0.786630 & -1.578318 \\\\\nStandard Deviation & 0.038050 & 0.025960 & 0.029236 & 0.009358 \\\\\nAnnualized Volatility & 0.604020 & 0.412100 & 0.464112 & 0.148557 \\\\\nMax Drawdown & 0.673269 & 0.125734 & 0.450001 & 0.159882 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [20]:
TSLA_full = pd.read_csv('TSLA-full.csv')
TSLA_full.to_latex(index=False)

'\\begin{tabular}{lrrrrrr}\n\\toprule\nUnnamed: 0 & Buy & Hold & FinMe & FinGPT & A2C & PPO & DQN \\\\\n\\midrule\nStandard Deviation & 0.039527 & 0.027419 & 0.031453 & 0.039521 & 0.039521 & 0.039382 \\\\\nAnnualized Volatility & 0.627470 & 0.435264 & 0.499305 & 0.627370 & 0.627374 & 0.625173 \\\\\nCumulative Return & 0.181865 & 0.989811 & 0.103947 & 0.230767 & 0.228670 & 0.701477 \\\\\nSharpe Ratio & 4.601053 & 36.099395 & 3.304806 & 5.839171 & 5.786047 & 17.812020 \\\\\nMax Drawdown & 0.553208 & 0.124524 & 0.423993 & 0.563435 & 0.525720 & 0.521396 \\\\\n\\bottomrule\n\\end{tabular}\n'